I created a new notebook because the one with XGBoost was requiring too much ram, this one is built not to study the model but to be compared with the FBT in terms of f1 scores

#NONOTEBOOK

In [ ]:
# just to be shure we will retrain the final model
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from xgboost import XGBClassifier # XGBoost classifier
from itertools import product #for grid search
from tqdm import tqdm # for progress bar
from sklearn.metrics import classification_report, f1_score # for evaluation
from utility.classification_utility import make_dataset_for_classification
from imblearn.over_sampling import RandomOverSampler # for oversampling
import json
f1_macro = lambda x, y: f1_score(x, y, average='macro') # for our evaluation

cyc_path = '../dataset/cyclists_cleaned.csv'
races_path = '../dataset/races_cleaned.csv'
print('Loading data...')
dataset = make_dataset_for_classification(races_df=races_path, cyclists_df=cyc_path)

with open('./XGB_rs_hyperparameters.json', 'r') as f:
    hyperparameterStore = json.load(f)

hyperparameterStore.sort(key=lambda x: x['score']['macro avg']['f1-score'], reverse=True)
best_params = hyperparameterStore[0]

TO_USE_COLS = [
    # over time
    'total_points', 'avg_points_per_race', 
    'average_position', 'avg_speed_cyclist',
    'race_count',
    'average_position_var',
    # race related
    'profile', 
    'startlist_quality', 'cyclist_age_rac', 'steepness', 
    'partecipants_number',
    # cyclist related
    'bmi',
]

tr_data = dataset[(dataset['date'] < '2019-01-01') & (dataset['date'] >= '1996-01-01')]
tr_out = tr_data['target']
tr_data = tr_data[TO_USE_COLS]

val_data = dataset[(dataset['date'] >= '2019-01-01') & (dataset['date'] < '2022-01-01')]
val_out = val_data['target']
val_data = val_data[TO_USE_COLS]

oversample = RandomOverSampler(
    sampling_strategy={
        0: 303478,
        1: 180000, 
    },

    random_state=42
)
tr_data, tr_out = oversample.fit_resample(tr_data, tr_out)
tr_data = tr_data.reset_index(drop=True)
tr_out = tr_out.reset_index(drop=True)
